---
# 고객 성향 그룹 클러스터링
---

1. 데이터 불러오기
    - customer_info : 고객 기본 정보
    - customer_type : 고객 성향 정보
    - customer_regis : 고객이 등록한 화장품 목록
    - product_info : 화장품 제품 정보
    - solvent : 화장품 성분별 정보

2. 고객의 성향 정보에 따른 클러스터링
    -> 클러스터링 결과로 고객이 어떤 그룹에 속하는지 확인

3. 성분 분석 : (고객이 속한 그룹군이 많이 사용하고 있는 성분)과 (고객이 갖고 있는 화장품의 성분) 비교
    - 성분 중 solvent, preservant, '정제수'는 제외
    - 성분명 자체에 쉼표가 표시된 성분 예외 처리 
    - 성분명 인코딩

# 데이터 준비

## 데이터 불러오기

##### 고객 성향 정보 : DB - customer_type 끌어오기

In [1]:
import pandas as pd
import pymysql

conn = pymysql.connect(host='localhost', user='root', password='apple123!!', db='beaulol', charset='utf8')

with conn.cursor() as curs:
    sql = "select * from customer_type"
    customer_type = pd.read_sql(sql, conn)

customer_type.head()

/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1205/941344558.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_type = pd.read_sql(sql, conn)


,userID,user_tone,user_skintype,user_trouble,user_style
0,F0002,가을웜톤,건성,민감성/아토피,데일리
1,F0003,겨울쿨톤,복합성,주름/탄력,유니크(힙)
2,F0004,봄웜톤,중성,잡티/미백/홍조/다크서클,러블리
3,F0005,겨울쿨톤,건성,각질,유니크(힙)
4,F0006,여름쿨톤,중성,트러블,데일리


In [2]:
customer_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   userID         9999 non-null   object
 1   user_tone      9999 non-null   object
 2   user_skintype  9999 non-null   object
 3   user_trouble   9999 non-null   object
 4   user_style     9999 non-null   object
dtypes: object(5)
memory usage: 390.7+ KB


##### 제품 정보 : DB - product_info 끌어오기

In [3]:
conn = pymysql.connect(host='localhost', user='root', password='apple123!!', db='beaulol', charset='utf8')

with conn.cursor() as curs:
    sql = "select * from product_info"
    product_info = pd.read_sql(sql, conn)

product_info

/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1205/3709294725.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product_info = pd.read_sql(sql, conn)


,prd_name,prd_brand,prd_price,prd_capa,prd_ingredients,prd_exp,prd_category
0,10% 나이아신아마이드 부스터 앰플,폴라초이스,"69,000",20ml,"정제수, 나이아신아마이드, 아세틸글루코사민, 아스코빌글루코사이드, 부틸렌글라이콜, ...",36개월 / 12개월,세럼
1,1025 독도 로션,라운드랩,"20,000",50ml,"정제수, 글리세린, 퀸즈랜드넛오일, 1,2-헥산다이올, 펜틸렌글라이콜, 호호바에스터...","제조일로부터 36개월, 개봉 후 12개월",로션
2,1025 독도 토너,라운드랩,"30,000",100ml,"정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물...","제조일로부터 36개월, 개봉 후 12개월 이내 사용",스킨토너
3,"11년간, 1등 잡티세럼",아이소이,"54,000",20ml+20ml+15ml,"정제수, 다마스크장미꽃수, 글리세린, 에탄올,알부틴, 베타인, 트레할로오스, 다마스...","제조일자로부터 3년, 개봉 후 6개월 내 사용권장",세럼
4,1번 진정 맑게담은 청초토너,넘버즈인,"40,000",300ml,"병풀추출물, 약모밀추출물, 감초뿌리추출물, 1,2-헥산다이올, 다이프로필렌글라이콜,...",[본품] 제조일로부터 36개월 / 개봉 후 12개월 이내 사용,스킨토너
...,...,...,...,...,...,...,...
432,히알루로닉 에센스,로벡틴,"29,000",180ml,"정제수, 글리세린, 메틸글루세스-20, 펜틸렌글라이콜, 부틸렌글라이콜, 나이아신아마...",개봉일로부터 1년 내,세럼
433,히알루론산 레이어 물톡스 앰플,메디필,"24,900",30ml,"정제수, 프로판다이올, 글리세린, 다이프로필렌글라이콜, 메틸프로판다이올, 1,2-헥...",개봉 전 30개월 / 개봉 후 12개월,세럼
434,히알루론산 레이어 물톡스 앰플 더블 기획,메디필,"34,000",30ml,"정제수, 프로판다이올, 글리세린, 다이프로필렌글라이콜, 메틸프로판다이올, 1,2-헥...",개봉 전 36개월 / 개봉 후 12개월,세럼
435,힐러 턴오버 액티브 크림,리쥬란,"49,000",50ml,"정제수, 글리세린, 부틸렌글라이콜, 메틸하이드로제네이티드로지네이트, 다이프로필렌글라...",제조일로부터 36개월 이내,크림


##### 성분 정보 : DB - solvent 끌어오기

In [4]:
conn = pymysql.connect(host='localhost', user='root', password='apple123!!', db='beaulol', charset='utf8')

with conn.cursor() as curs:
    sql = "select * from solvent"
    solvent = pd.read_sql(sql, conn)
solvent

/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1205/2397109920.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  solvent = pd.read_sql(sql, conn)


,ingredient_eng,ingredient_kor
0,CYCLOTETRASILOXANE,사이클로테트라실록세인
1,TETRADECENE,테트라데센
2,METHYL TRIMETHICONE,메틸트라이메티콘
3,OCTADECANE,옥타데케인
4,PEG-18,피이지-18
...,...,...
596,EGG SHELL MEMBRANE EXTRACT,난각막추출물
597,O-CYMEN-5-OL,o-사이멘-5-올
598,POTASSIUM METABISULFITE,포타슘메타바이설파이트
599,HYDROGENATED TALLOWTRIMONIUM CHLORIDE,하이드로제네이티드탈로우트라이모늄클로라이드


In [5]:
# 성분명 자체에 쉼표가 표시된 성분들 -> 추후 예외 처리 필요
filtered_data = list(solvent[solvent['ingredient_kor'].str.contains(',')]['ingredient_kor'].values)
filtered_data

['1,2,6-헥산트라이올',
 '1,2-헥산다이올',
 '1,2-부탄다이올',
 '1,4-부탄다이올',
 '1,5-펜탄다이올',
 '트라이메틸-1,3-펜탄다이올',
 '2,3-부탄다이올',
 '다이에틸헥실2,6-나프탈레이트',
 '1,10-데칸다이올',
 '폴리(1,2-부탄다이올)-6프로필렌글라이콜',
 '2-브로모-2-나이트로프로판-1,3-디올',
 '5-브로모-5-나이트로-1,3-디옥산']

##### 고객 등록 제품 정보 : DB - customer_regis 끌어오기

In [6]:
conn = pymysql.connect(host='localhost', user='root', password='apple123!!', db='beaulol', charset='utf8')

with conn.cursor() as curs:
    sql = "select * from customer_regis"
    customer_regis = pd.read_sql(sql, conn)
customer_regis

/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1205/365437681.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_regis = pd.read_sql(sql, conn)


,regis_num,regis_date,userID,prd_brand,prd_name,pay_date,pay_price,op_date
0,240123_F0002_00,2024-01-23,F0002,얼트루,아임쏘리포마이스킨 릴렉싱 앰플,None,None,None
1,240123_F0002_01,2024-01-23,F0002,닥터지,에이클리어 스팟 포 페이스 세럼,None,None,None
2,240123_F0002_02,2024-01-23,F0002,라운드랩,1025 독도 토너,None,None,None
3,240123_F0002_03,2024-01-23,F0002,아벤느,트릭세라 로션,None,None,None
4,240123_F0002_04,2024-01-23,F0002,에스트라,아토베리어365 하이드로 수딩크림,None,None,None
...,...,...,...,...,...,...,...,...
49990,240123_M9994_00,2024-01-23,M9994,메디필,히알루론산 레이어 물톡스 앰플 더블 기획,None,None,None
49991,240123_M9994_01,2024-01-23,M9994,닐스야드 레머디스,레머디스 와일드 로즈 뷰티 세럼,None,None,None
49992,240123_M9994_02,2024-01-23,M9994,닥터지,더모이스처 배리어 D 리퀴드,None,None,None
49993,240123_M9994_03,2024-01-23,M9994,아크네스,모이스처 플루이드,None,None,None


# 전처리
- userID, userID별 피부타입, 피부 고민, 등록 화장품, 등록 화장품의 성분 합치기

### 필요한 칼럼 합친 데이터프레임 생성

In [7]:
df_prep = pd.DataFrame()
df_prep

""


In [8]:
type_prep = customer_type[['userID','user_skintype','user_trouble']]
type_prep

,userID,user_skintype,user_trouble
0,F0002,건성,민감성/아토피
1,F0003,복합성,주름/탄력
2,F0004,중성,잡티/미백/홍조/다크서클
3,F0005,건성,각질
4,F0006,중성,트러블
...,...,...,...
9994,M9986,민감성,각질
9995,M9991,지성,트러블
9996,M9992,민감성,잡티/미백/홍조/다크서클
9997,M9993,건성,잡티/미백/홍조/다크서클


In [9]:
grouped = customer_regis.groupby('userID')['prd_name'].apply('&&'.join).reset_index()
grouped.rename(columns={'prd_name': 'customer_prd'}, inplace=True)

df_prep = customer_type[['userID','user_skintype','user_trouble']].merge(grouped, on='userID', how='left')
df_prep

,userID,user_skintype,user_trouble,customer_prd
0,F0002,건성,민감성/아토피,아임쏘리포마이스킨 릴렉싱 앰플&&에이클리어 스팟 포 페이스 세럼&&1025 독도 토...
1,F0003,복합성,주름/탄력,하이드라메모리 세럼&&멀티-펩타이드 + 카퍼 펩타이즈 1% 세럼&&크림스킨&&포 맨...
2,F0004,중성,잡티/미백/홍조/다크서클,아하 7 화이트헤드 파워 리퀴드&&레티놀 흔적 리페어 앰플&&1번 진정 맑게담은 청...
3,F0005,건성,각질,파이브 플라워 다크스팟 클리어링 세럼&&브링그린 토닝비타20%빛세럼25mL&&화이트...
4,F0006,중성,트러블,로얄 블랙 스네일 퍼스트 에센스&&모링가 타이트닝 포어 세럼&&레드수딩 시카 밸런스...
...,...,...,...,...
9994,M9986,민감성,각질,레드블레미쉬 클리어 수딩 액티브 에센스&&뮤제너 피토 수딩 앰플 대용량 기획&&다이...
9995,M9991,지성,트러블,솔루션 10%&&레티놀 0.5% 인 스쿠알란&&1025 독도 토너&&레드 블레미쉬 ...
9996,M9992,민감성,잡티/미백/홍조/다크서클,라운드랩 1025 독도 앰플 45g&&온리 포 맨 올인원에센스&&더모이스처 배리어 ...
9997,M9993,건성,잡티/미백/홍조/다크서클,에이지알 글루타치온 글로우 앰플&&아쿠아 이온 플라즈마 수분 앰플&&하이드라비오 토...


### 유저가 갖고 있는 화장품들의 성분 가져오기

#### 1.성분명에 콤마 있는 성분들 *로 변환

In [10]:
def remove_comma_name(df,columns,filter_list):
    for column in df.columns:
        for filter in filter_list:
            df[column] = df[column].apply(lambda x : x.replace(filter,filter.replace(",","!!")) if filter in x else x)
    return df

In [11]:
def remove_marks(df,columns,filter_list):
    for column in df.columns:
        for filter in filter_list:
            df[column] = df[column].apply(lambda x : x.replace(filter,filter.replace("@",",")) if filter in x else x)
            df[column] = df[column].apply(lambda x : x.replace(filter,filter.replace("*","")) if filter in x else x)
            df[column] = df[column].apply(lambda x : x.replace(filter,filter.replace("■ ","")) if filter in x else x)
    return df

##### solvent_prep df :solvent 성분명들 중에 포함된 쉼표 제거

In [12]:
filter_comma = list(solvent[solvent['ingredient_kor'].str.contains(',')]['ingredient_kor'].values)
solvent_prep = remove_comma_name(solvent,solvent.columns,filter_comma)
solvent_prep

,ingredient_eng,ingredient_kor
0,CYCLOTETRASILOXANE,사이클로테트라실록세인
1,TETRADECENE,테트라데센
2,METHYL TRIMETHICONE,메틸트라이메티콘
3,OCTADECANE,옥타데케인
4,PEG-18,피이지-18
...,...,...
596,EGG SHELL MEMBRANE EXTRACT,난각막추출물
597,O-CYMEN-5-OL,o-사이멘-5-올
598,POTASSIUM METABISULFITE,포타슘메타바이설파이트
599,HYDROGENATED TALLOWTRIMONIUM CHLORIDE,하이드로제네이티드탈로우트라이모늄클로라이드


##### prd_info_prep df : product_info 화장품 성분들 중 이름에 콤마 들어있는 성분 포함된 경우 처리

In [13]:
prd_info_prep = remove_comma_name(product_info,product_info['prd_ingredients'],filter_comma)[['prd_name','prd_ingredients']]
prd_info_prep

,prd_name,prd_ingredients
0,10% 나이아신아마이드 부스터 앰플,"정제수, 나이아신아마이드, 아세틸글루코사민, 아스코빌글루코사이드, 부틸렌글라이콜, ..."
1,1025 독도 로션,"정제수, 글리세린, 퀸즈랜드넛오일, 1!!2-헥산다이올, 펜틸렌글라이콜, 호호바에스..."
2,1025 독도 토너,"정제수, 부틸렌글라이콜, 글리세린, 펜틸렌글라이콜, 프로판다이올, 아이리쉬모스추출물..."
3,"11년간, 1등 잡티세럼","정제수, 다마스크장미꽃수, 글리세린, 에탄올,알부틴, 베타인, 트레할로오스, 다마스..."
4,1번 진정 맑게담은 청초토너,"병풀추출물, 약모밀추출물, 감초뿌리추출물, 1!!2-헥산다이올, 다이프로필렌글라이콜..."
...,...,...
432,히알루로닉 에센스,"정제수, 글리세린, 메틸글루세스-20, 펜틸렌글라이콜, 부틸렌글라이콜, 나이아신아마..."
433,히알루론산 레이어 물톡스 앰플,"정제수, 프로판다이올, 글리세린, 다이프로필렌글라이콜, 메틸프로판다이올, 1!!2-..."
434,히알루론산 레이어 물톡스 앰플 더블 기획,"정제수, 프로판다이올, 글리세린, 다이프로필렌글라이콜, 메틸프로판다이올, 1!!2-..."
435,힐러 턴오버 액티브 크림,"정제수, 글리세린, 부틸렌글라이콜, 메틸하이드로제네이티드로지네이트, 다이프로필렌글라..."


##### 유저 등록 화장품들 성분 가져오기

In [14]:
# # 유저 등록 화장품들 성분 가져오기
# products = df_prep['customer_prd'][0].split('&&')        # 합친 제품들 제품 하나하나씩 쪼개기
# prd_ingredients = []
# for idx,product in enumerate(products):                            # 쪼갠 제품 이름 하나씩 출력
#     if product != "":
#         prd_ingredients.append(product_info.loc[product_info['prd_name']==product]['prd_ingredients'].item())
#         for ingredient in prd_ingredients:
#             print(ingredient)
            
        

In [15]:
# prd_ingredients[0].split(",")[0]

In [16]:
# product = products[0]
# test = product_info.loc[product_info['prd_name']==product]['prd_ingredients'].item().split("&&")
# test

In [17]:
# products = df_prep['customer_prd'][0].split('&&')        # 합친 제품들 제품 하나하나씩 쪼개기
# prd_ingredients = []
# user_ingredients = []
# for idx,product in enumerate(products):                            # 쪼갠 제품 이름 하나씩 출력
#     product_ingredients = []
#     if product != "":
#         prd_ingredients.append(product_info.loc[product_info['prd_name']==product]['prd_ingredients'].item())
#         for ingredients in prd_ingredients:
#             for ingredient in ingredients.split(", "):
#                 product_ingredients.append(ingredient)
#                 user_ingredients.append(ingredient)
#         print(product,product_ingredients)
# print()
# print(user_ingredients)

In [18]:
# user_ingredients_test = ",".join(user_ingredients)
# user_ingredients_test

## 성분 중 불필요한 정제수, 용매, 방부제 삭제

In [19]:
filter_solvent = list(solvent_prep['ingredient_kor'].values)
filter_solvent

['사이클로테트라실록세인',
 '테트라데센',
 '메틸트라이메티콘',
 '옥타데케인',
 '피이지-18',
 '사이클로헥사실록세인',
 '피이지/피피지-6/2코폴리머',
 '에틸에터',
 '헥산다이올',
 '다이부틸아디페이트',
 '다이메틸아이소소바이드',
 '하이드로제네이티드폴리데센',
 '피피지-55글리세릴에터',
 '퍼플루오로퍼하이드로페난트렌',
 '아이소옥탄',
 '퍼플루오로다이메틸사이클로헥산',
 '페녹시프로판다이올',
 '다이옥솔레인',
 '사이클로펜타실록세인',
 '3-메톡시부탄올',
 '트라이부틸시트레이트',
 '상어간오일',
 '다이에틸옥살레이트',
 '헥실렌글라이콜',
 '다이에톡시에틸석시네이트',
 '에틸아세테이트',
 '트라이아세틴',
 '트라이에틸포스페이트',
 '벤질벤조에이트',
 '프로필렌글라이콜부틸에터',
 '다이메틸석시네이트',
 '엠이케이',
 '글리세레스-18벤조에이트',
 '자일렌',
 '펜타플루오로프로판',
 '메톡시아이소프로판올',
 '스쿠알렌',
 '다이아이소옥틸아디페이트',
 '피피지-3',
 'C16-17알킬벤조에이트',
 '미비케이',
 'C10-12알케인/사이클로알케인',
 '사이클로헥산온',
 '다이-피피지-3미리스틸에터아디페이트',
 '피이지/피피지-200/70코폴리머',
 'C14-19알케인',
 '톨루엔',
 '솔베스-30',
 '테트라하이드로퍼퓨릴아세테이트',
 'C10-18트라이글리세라이즈',
 '다이에틸세바케이트',
 '다이부틸옥살레이트',
 '아이소부틸벤조에이트',
 '퍼플루오로데칼린',
 '옥틸도데칸올',
 '다이메틸프탈레이트',
 '피이지-4',
 '피이지-240',
 '퍼플루오로퍼하이드로벤질테트랄린',
 '글리세레스-7락테이트',
 '메틸부텐',
 '피피지-7',
 '피이지/피피지-25/30코폴리머',
 '피이지-135',
 '에틸피롤리돈',
 '스테아릴벤조에이트',
 '다이트라이데실아디페이트',
 '데카플루오로펜탄',
 '사이클로헵타실록세인',
 '글라이콜',
 '스타이렌',
 '피이지-60

In [20]:
# products = df_prep['customer_prd'][0].split('&&')        # 합친 제품들 제품 하나하나씩 쪼개기
# prd_ingredients = []
# for idx,product in enumerate(products):                            # 쪼갠 제품 이름 하나씩 출력
#     product_ingredients = []
#     if product != "":
#         prd_ingredients.append(product_info.loc[product_info['prd_name']==product]['prd_ingredients'].item())
#         for ingredients in prd_ingredients:
#             for ingredient in ingredients.split(", "):
#                 if ingredient not in filter_solvent:
#                     product_ingredients.append(ingredient)
#         print(product,product_ingredients)

# 최종 전처리

In [42]:
for i,user in enumerate(df_prep['userID']):
    products = df_prep['customer_prd'][i].split('&&')        # 합친 제품들 제품 하나하나씩 쪼개기
    user_ingredients = []
    prd_ingredients = []
    for idx,product in enumerate(products):                            # 쪼갠 제품 이름 하나씩 출력
        if product != "":
            prd_ingredients.append(product_info.loc[product_info['prd_name']==product]['prd_ingredients'].item())   # 제품 하나의 성분 정보 담기
    for x,ingredients in enumerate(prd_ingredients):
        # print("___")
        # print(ingredients)
        for ingredient in ingredients.split(","):
            if "(" in ingredient:
                ingredient = ingredient.split("(")[0].strip()
            if "*" in ingredient:
                ingredient = ingredient.split("*")[0].strip()       #
                ingredient = ingredient.replace("*","")             # 다시 생각해보기
            if "정제수" in ingredient:
                ingredient = '정제수'
            if (ingredient.strip() not in filter_solvent) & ("ppm" not in ingredient) & ("ppb" not in ingredient) & (")" not in ingredient) & ("%" not in ingredient):
                user_ingredients.append(ingredient.strip())
    #             print(user_ingredients,"1")
    # print(user_ingredients,"2")
    df_prep.loc[i,'user_ingredients'] = ",".join(user_ingredients)


In [41]:
df_prep['user_ingredients'][0]

'병풀잎수,메틸글루세스-20,폴리글리세린-3,병풀추출물,무화과추출물,다시마추출물,한련초잎추출물,버지니아풍년화추출물,티트리잎추출물,약모밀추출물,국화추출물,연꽃꽃추출물,라벤더꽃추출물,마돈나백합꽃추출물,서양수수꽃다리추출물,장미꽃추출물,녹차수,베르가모트오일,하이드로제네이티드레시틴,C12-14파레스-12,알란토인,마트리카리아꽃수,잔탄검,프룩토올리고사카라이드,베타-글루칸,하이드롤라이즈드하이알루로닉애씨드,세라마이드엔피,카보머,팔미토일펜타펩타이드-4,에틸헥실글리세린,리날룰,나이아신아마이드,락토바실러스발효물,암모늄폴리아크릴로일다이메틸타우레이트,판테놀,석시닉애씨드,트로메타민,다이포타슘글리시리제이트,알란토인,스테비아잎/줄기추출물,다이소듐이디티에이,덱스트린,카카오씨추출물,병풀추출물,약모밀추출물,워터민트추출물,애플민트잎추출물,스피어민트추출물,아시아틱애씨드,아시아티코사이드,마데카식애씨드,마데카소사이드,흰버드나무껍질추출물,아이리쉬모스추출물,사탕수수추출물,해수,프로테아제,베타인,판테놀,에틸헥실글리세린,알란토인,잔탄검,다이소듐이디티에이,아벤느온천수,카프릴릭/카프릭트리글리세라이드,이소프로필팔미테이트,프로필렌글라이콜디카프레이트,글리세릴스테아레이트,피이지-100스테아레이트,미레스-3미리스테이트,토코페릴아세테이트,달맞이꽃오일,카프릴릴글라이콜,글라이신,펜틸람노사이드,아크릴레이트/C10-30알킬아크릴레이트크로스폴리머,카보머,소듐하이드록사이드,향료,콩추출물,토코페롤,페닐트라이메티콘,트레할로오스,다이카프릴릴카보네이트,암모늄아크릴로일다이메틸타우레이트/브이피코폴리머,하이드로제네이티드레시틴,하이드로제네이티드폴리아이소부텐,하이드록시에틸아크릴레이트/소듐아크릴로일다이메틸타우레이트코폴리머,세테아릴알코올,아크릴레이트/C10-30알킬아크릴레이트크로스폴리머,글리세릴카프릴레이트,글리세릴스테아레이트,트로메타민,다이소듐이디티에이,에틸헥실글리세린,스테아릭애씨드,팔미틱애씨드,폴리글리세릴-3메틸글루코오스다이스테아레이트,솔비탄아이소스테아레이트,만니톨,알지닌,세린,수크로오스,피씨에이,시트룰린,글라이코젠,히스티딘에이치씨엘,알라닌,토코페롤

In [43]:
df_prep

,userID,user_skintype,user_trouble,customer_prd,user_ingredients
0,F0002,건성,민감성/아토피,아임쏘리포마이스킨 릴렉싱 앰플&&에이클리어 스팟 포 페이스 세럼&&1025 독도 토...,"병풀잎수,메틸글루세스-20,폴리글리세린-3,병풀추출물,무화과추출물,다시마추출물,한련..."
1,F0003,복합성,주름/탄력,하이드라메모리 세럼&&멀티-펩타이드 + 카퍼 펩타이즈 1% 세럼&&크림스킨&&포 맨...,"스쿠알란,하이드롤라이즈드효모추출물,하이드록시에틸아크릴레이트/소듐아크릴로일다이메틸타우..."
2,F0004,중성,잡티/미백/홍조/다크서클,아하 7 화이트헤드 파워 리퀴드&&레티놀 흔적 리페어 앰플&&1번 진정 맑게담은 청...,"사과수,글라이콜릭애씨드,나이아신아마이드,소듐하이드록사이드,판테놀,소듐하이알루로네이트..."
3,F0005,건성,각질,파이브 플라워 다크스팟 클리어링 세럼&&브링그린 토닝비타20%빛세럼25mL&&화이트...,"글리세레스-26,나이아신아마이드,아크릴레이트/C10-30알킬아크릴레이트크로스폴리머,..."
4,F0006,중성,트러블,로얄 블랙 스네일 퍼스트 에센스&&모링가 타이트닝 포어 세럼&&레드수딩 시카 밸런스...,"달팽이점액여과물,나이아신아마이드,판테놀,자이모모나스발효추출물,꿀추출물,하이드롤라이즈..."
...,...,...,...,...,...
9994,M9986,민감성,각질,레드블레미쉬 클리어 수딩 액티브 에센스&&뮤제너 피토 수딩 앰플 대용량 기획&&다이...,"약모밀추출물,병풀추출물,케이프알로에잎추출물,나이아신아마이드,엑토인,알란토인,아크릴레..."
9995,M9991,지성,트러블,솔루션 10%&&레티놀 0.5% 인 스쿠알란&&1025 독도 토너&&레드 블레미쉬 ...,"아세틸헥사펩타이드-8,트라이소듐에틸렌다이아민다이석시네이트,젤란검,소듐클로라이드,아이..."
9996,M9992,민감성,잡티/미백/홍조/다크서클,라운드랩 1025 독도 앰플 45g&&온리 포 맨 올인원에센스&&더모이스처 배리어 ...,"해수,하이드롤라이즈드하이알루로닉애씨드,하이드롤라이즈드콜라겐,판테놀,아이리쉬모스추출물..."
9997,M9993,건성,잡티/미백/홍조/다크서클,에이지알 글루타치온 글로우 앰플&&아쿠아 이온 플라즈마 수분 앰플&&하이드라비오 토...,"나이아신아마이드,베타인,글리세레스-26,비스-피이지-18메틸에터다이메틸실레인,폴리글..."


In [44]:
df_prep.to_csv('./output/clustering_prep06.csv',index=False)
df_prep.to_excel('./output/clustering_prep06.xlsx',index=False)